In [1]:
import sqlite3

In [2]:
connection = sqlite3.connect('db.sqlite3')
cursor = connection.cursor()

In [3]:
cursor.execute('SELECT name FROM sqlite_master WHERE type="table";')
print(cursor.fetchall())

[('django_migrations',), ('sqlite_sequence',), ('django_admin_log',), ('django_content_type',), ('auth_group_permissions',), ('auth_permission',), ('auth_group',), ('django_session',), ('django_site',), ('user_servicecategory',), ('user_customuser',), ('user_business',), ('user_service',), ('user_booking',)]


In [ ]:
cursor.execute('SELECT * FROM user_customuser;')
print(cursor.fetchall())

In [3]:
#connection.commit()
cursor.close()
connection.close()